This notebook can be run on mybinder: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/git/https%3A%2F%2Fgricad-gitlab.univ-grenoble-alpes.fr%2Fchatelaf%2Fconference-ia/master?urlpath=lab/tree/notebooks/5_classification/N3_importance_of_scaling-svm.ipynb)

In [1]:
# Import modules
%matplotlib inline
import matplotlib
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Generate (unscaled) data

In [2]:
sp.random.seed(10)
X = sp.random.randn(200,2)
X[:,0] *= 0.1
X[:,1] *= 100

# Create a vertical class boundary: depends on the first variable X[:,0]  only
y = sp.ones((200,))
y[X[:,0]<0]=0

# Plot
plt.scatter(X[:,0], X[:,1], c=y)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 Here the second variable is **three magnitude orders larger** than the first one
 
 We train now a SVM classifier on these unscaled data

In [3]:
# Performance of SVM  (rbf kernel) classifier

from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=0, stratify=y)
model = SVC(kernel='rbf',gamma='auto')
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("Classification accuracy before scaling: {}".format(accuracy_score(y_test,y_pred)))

Classification accuracy before scaling: 0.5625


In [4]:
# display decision boundary

from matplotlib.colors import ListedColormap

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = sp.meshgrid(sp.arange(x_min, x_max, (x_max-x_min)/1000),
                         sp.arange(y_min, y_max,(y_max-y_min)/1000 ))
# just plot the dataset first
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

# Plot the training points
fig, ax= plt.subplots()
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
    edgecolors='k')

ax.set_xlim(xx.min(), xx.max())
ax.set_ylim(yy.min(), yy.max())
ax.set_xticks(())
ax.set_yticks(())

Z = model.decision_function(sp.c_[xx.ravel(), yy.ravel()])
CS = ax.contour(xx, yy, Z.reshape(xx.shape), [0 ]) 
ax.clabel(CS, inline=1, fontsize=10)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Scale data

### Minmax scaling

In [11]:
 # Scale data between 0 and 1
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X_train_s= sc.fit_transform(X_train) # Scale data between 0 and 1
X_test_s= sc.transform(X_test) # Scale data between 0 and 1

In [12]:
# We train now a SVM classifier on these SCALED data

model = SVC(kernel='rbf',gamma='auto')
model.fit(X_train_s,y_train)
y_pred = model.predict(X_test_s)
print("Classification accuracy after scaling: {}".format(accuracy_score(y_test,y_pred)))

Classification accuracy after scaling: 0.9375


### Standardization scaling

In [13]:
# Standardize the data (centered with unit variance)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_s= sc.fit_transform(X_train)
X_test_s= sc.transform(X_test)

model = SVC(kernel='rbf',gamma='auto')
model.fit(X_train_s, y_train)
y_pred = model.predict(X_test_s)
print("Classification accuracy after standardization: {}".format(accuracy_score(y_test,y_pred)))

Classification accuracy after standardization: 0.9625


In [14]:
# display decision boundary
from matplotlib.colors import ListedColormap

x_min, x_max = X_train_s[:, 0].min() - .5, X_train_s[:, 0].max() + .5
y_min, y_max = X_train_s[:, 1].min() - .5, X_train_s[:, 1].max() + .5
xx, yy = sp.meshgrid(sp.arange(x_min, x_max, (x_max-x_min)/100),
                         sp.arange(y_min, y_max,(y_max-y_min)/100 ))
# just plot the dataset first
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

# Plot the training points
fig, ax= plt.subplots()
plt.scatter(X_train_s[:, 0], X_train_s[:, 1], c=y_train, cmap=cm_bright,
    edgecolors='k')
ax.set_xlim(xx.min(), xx.max())
ax.set_ylim(yy.min(), yy.max())
ax.set_xticks(())
ax.set_yticks(())

Z = model.decision_function(sp.c_[xx.ravel(), yy.ravel()])
CS = ax.contour(xx, yy, Z.reshape(xx.shape), [-1, 0,1 ]) 
ax.clabel(CS, inline=1, fontsize=10)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Using a different kernel, for instance a linear kernel

In [24]:
model = SVC(kernel='poly', gamma="auto", degree=1, C=10)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("Classification accuracy before scaling with polynomial kernel: {}".format(accuracy_score(y_test,y_pred)))

Classification accuracy before scaling with polynomial kernel: 0.9


In [20]:
model = SVC(kernel='poly', gamma="auto", degree=1, C=1)
model.fit(X_train_s,y_train)
y_pred = model.predict(X_test_s)
print("Classification accuracy after scaling with polynomial kernel: {}".format(accuracy_score(y_test,y_pred)))

Classification accuracy after scaling with polynomial kernel: 0.95


### Exercice
- What happen for the lineat kernel when $C=1$?
- Can you explain why in this case the accuracy is improved by increasing the cost parameter $C$ (even for unscaled data)?
- Add some noise (e.g. change the label for some data points) add check if it still works (good accuray for unscaled data with large C)


In [22]:
y_train[:10]= 1-y_train[:10] 

In [23]:
y_train[:10]

array([1., 1., 0., 1., 1., 0., 1., 0., 0., 1.])